In [1]:
from acfunctions import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.stats as astat
from solartwins import *
import pickle
from IPython.display import Image, display, HTML
import starspot as ss
import starspot.rotation_tools as rt
from starspot import sigma_clipping
import lightkurve as lk
from lightkurve import search_targetpixelfile, search_lightcurvefile
import math
import exoplanet
import eleanor
from astropy.coordinates import SkyCoord
from scipy import interpolate
from tess_kep_funct import *
from tqdm import tqdm
from scipy import misc
from bokeh.models import ColumnDataSource, LabelSet, Whisker
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.sampledata.autompg import autompg as df

import warnings
warnings.filterwarnings("ignore")

/Users/ilanadoran/opt/anaconda3/lib/python3.7/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=False)

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
with open('tess_period.data', 'rb') as filehandle:
    short_tess_period = pickle.load(filehandle) 

In [5]:
with open('tess_periods_interp.data', 'rb') as filehandle:
    tess_periods_interp = pickle.load(filehandle) 

In [6]:
with open('ls_periods_interp.data', 'rb') as filehandle:
    ls_period_interp = pickle.load(filehandle) 

In [7]:
with open('ls_period_no_interp.data', 'rb') as filehandle:
    ls_period_no_interp = pickle.load(filehandle) 

In [8]:
#dataset with tess vs kepler IDs
tess_kep = pd.read_csv('data_summary.csv')
tess_short = tess_kep[tess_kep.tess_ffi == True]
cat = pd.read_csv('tics_kics_mcq_rot.txt', names=['tic_id', 'kic_id', 'prot'], delim_whitespace=True)

In [9]:
#importing McQ periods
headers = 'TIC ID', 'KIC ID', 'McQ Periods';  

with open("tics_kics_mcq_rot.txt") as file:
    rot_table = [line.split() for line in file.read().splitlines()]

widths = [max(len(value) for value in col)
for col in zip(*(rot_table + [headers]))]

formatting = '{:{widths[0]}}  {:{widths[1]}}  {:{widths[2]}}'
formatting.format(*headers, widths=widths)
for tics_kics_mcq in rot_table:
    formatting.format(*tics_kics_mcq, widths=widths)

In [10]:
# converting txt file data to arrays
tic_ids = []
mcq_periods = []

for index, row in enumerate(rot_table):
    tic_ids.append(int(rot_table[index][0]))
    mcq_periods.append(float(rot_table[index][2]))

In [11]:
# selecting shortest periods only (<13 days, half a tess cycle)
short_periods = []
tics_short_periods = []

for index, period in enumerate(mcq_periods):
    if period < 13: 
        short_periods.append(period)
        tics_short_periods.append(tic_ids[index])
        
#limiting to the first 60
#short_periods = short_periods[0:61]
#tics_short_periods = tics_short_periods[0:61]

In [12]:
def find_m_b(x,y,err): #analytical approach to finding line of best fit 
    #C     
    errorsq = np.square(err)
    C = np.diag(errorsq)

    #A 
    xb = ([1] * len(x))
    mata = []
    for z, txt in enumerate(x):
        mata.append(x[z])
        mata.append(xb[z])
    A= np.matrix(mata).reshape((len(x), 2))

    #plugging in 
    At = np.transpose(A)
    invC = np.linalg.inv(C)
    pt1 = np.dot(At, np.dot(invC,A))
    invpt1= np.linalg.inv(pt1)
    pt2 = np.dot(At, np.dot(invC, y)).T
    cov = np.dot(invpt1, pt2)

    m = float(cov[0])
    b = float(cov[1])
    return m ,b 

def residuals(x, y, error): #residual abundance function
    mborig = find_m_b(x, y, error)
    m = mborig[0]
    b = mborig[1]

    predicted_values = [] #y values from slope
    pv = 0
    for u in x:
        pv = (m*u) + b
        predicted_values.append(pv)
        pv = 0

    prev = np.array(predicted_values)
    abu = np.array(y)
    diff = abu - prev #difference between slope and measured values  
    return diff

# test 

star = eleanor.Source(tic=tics_short_periods[0])
data = eleanor.TargetData(star, do_psf=True, do_pca=True)
q = data.quality == 0

time = data.time[q]    
norm_flux = data.pca_flux[q]/np.nanmedian(data.pca_flux[q])
flux_err = np.sqrt(norm_flux)

plt.plot(time, norm_flux, c = 'steelblue')
plt.ylabel('Normalized Flux')
plt.xlabel('Time')

find_m_b(time, norm_flux, flux_err)

resid = residuals(time, norm_flux, flux_err)
resid

plt.plot(time, resid)

#linear interpolation
f = interpolate.interp1d(time, resid)
xnew = np.arange(time[0], time[-1], 0.1)
flux = f(xnew) 
flux_err = np.sqrt(flux)
        
rotate_el = ss.RotationModel(xnew, flux, flux_err)
ls_period = rotate_el.ls_rotation()
    
period_grid = np.linspace(.1, 10, 1000)
pdm_period, period_err = rotate_el.pdm_rotation(period_grid, pdm_nbins=10)
pdm_plot = rotate_el.pdm_plot();
    
print(ls_period, pdm_period)

short_periods[0]

# all tess stars 

ls_periods_resid = []
pdm_periods_resid = []
pdm_error = []
exo_periods_resid = []
error = []
exo_error = []

for each, tic in enumerate(tics_short_periods[0:61]): 
    star = eleanor.Source(tic=tic)
    data = eleanor.TargetData(star, do_psf=True, do_pca=True)
    index = np.where(np.array(tics_short_periods) == tic)
    mcq_period = short_periods[index[0][0]]
    
    q = data.quality == 0

    time = data.time[q]    
    norm_flux = data.pca_flux[q]/np.nanmedian(data.pca_flux[q])
    norm_flux_err = np.sqrt(norm_flux)
    
    resid = residuals(time, norm_flux, norm_flux_err)
    resid = resid[~np.isnan(resid)]
    time = time[~np.isnan(resid)]
    
    try :
        #linear interpolation
        f = interpolate.interp1d(time, resid)
        xnew = np.arange(time[0], time[-1], 0.1)
        flux = f(xnew) 
        flux_err = np.sqrt(flux)
        
        #plot - flux
        #fig, axs = plt.subplots(2, figsize = (12,10))
        #fig.tight_layout(h_pad=6)
        
        #axs[0].plot(xnew, flux, c='steelblue')
        #axs[0].annotate('McQ Period = {0}'.format(mcq_period),
            #xy=(np.average(time), np.average(flux)), xycoords='data',
            #xytext=(0.1, 0.2), textcoords='offset points')
        #axs[0].set_title('Normalized LC for {}'.format(tic))
        #axs[0].set_xlabel('Time')
        #axs[0].set_ylabel('Flux')
    
        #plot - exoplanet's ac function
        ex_1 = exoplanet.autocorr_estimator(xnew, flux, yerr=flux_err, min_period = 0.1, max_period = 15)
        exo_period = ex_1['peaks'][0]['period']
        #axs[1].plot(ex_1['autocorr'][0], ex_1['autocorr'][1])
        #axs[1].set_title('Autocorrelation for {0}'.format(tic))
        #axs[1].annotate('McQ Period = {0}'.format(mcq_period),
            # xy=(np.average(ex_1['autocorr'][0]), np.average(ex_1['autocorr'][1])), xycoords='data',
            # xytext=(0.1, 0.2), textcoords='offset points')
        #axs[1].annotate('Peak = {0}'.format(np.around(ex_1['peaks'][0]['period'], 3)),
            #xy=(10, 0.5), xycoords='data',
            #xytext=(0.1, 0.2), textcoords='offset points')
        ex_error = ex_1['peaks'][0]['period_uncert']
        exo_error.append(ex_error)   
        exo_periods_resid.append(exo_period)
        
        #fig.savefig('tess_{0}_flux_ac.jpg'.format(each))
        #plt.close(fig)
        
        #plot - lomb-scargle 
        rotate_el = ss.RotationModel(xnew, flux, flux_err)
        ls_period = rotate_el.ls_rotation()
        #ls_plot = rotate_el.ls_plot()
        #plt.savefig('tess_{0}_ls.jpg'.format(each))
        #plt.close()

        
        #plot - pdm 
        period_grid = np.linspace(.1, 10, 1000)
        pdm_period, period_err = rotate_el.pdm_rotation(period_grid, pdm_nbins=10)
        #pdm_plot = rotate_el.pdm_plot()
        #plt.savefig('tess_{0}_pdm.jpg'.format(each), dpi = 75)
        #plt.close()
        
        ls_periods_resid.append(ls_period)
        pdm_periods_resid.append(pdm_period)
        pdm_error.append(period_err)
        print('works!')
        
        
    except : 
        print('error')
        error.append(each)
        ls_periods_resid.append(0)
        pdm_periods_resid.append(0)
        exo_periods_resid.append(0)
        exo_error.append(0)
        #time = data.time[q]    
        #norm_flux = data.pca_flux[q]/np.nanmedian(data.pca_flux[q])
        #norm_flux_err = np.sqrt(norm_flux)
        
        #plt.plot(time, norm_flux)
        #plt.xlabel('Time')
        #plt.ylabel('Normalized Flux')
        
        #plt.savefig('error_tess_{0}_flux_ac.jpg'.format(each))
        #plt.close()

with open('ls_periods_resid.data', 'wb') as filehandle:
    pickle.dump(ls_periods_resid, filehandle)

with open('pdm_periods_resid.data', 'wb') as filehandle:
    pickle.dump(pdm_periods_resid, filehandle)

with open('exo_periods_resid.data', 'wb') as filehandle:
    pickle.dump(exo_periods_resid, filehandle)

with open('pdm_error.data', 'wb') as filehandle:
    pickle.dump(pdm_error, filehandle)

In [13]:
with open('pdm_periods_resid.data', 'rb') as filehandle:
    pdm_periods_resid = pickle.load(filehandle) 

In [14]:
with open('ls_periods_resid.data', 'rb') as filehandle:
    ls_periods_resid = pickle.load(filehandle) 

In [15]:
with open('exo_periods_resid.data', 'rb') as filehandle:
    exo_periods_resid = pickle.load(filehandle) 

In [16]:
with open('pdm_error.data', 'rb') as filehandle:
    pdm_error = pickle.load(filehandle) 

In [17]:
with open('mcq_exo_difference.data', 'rb') as filehandle:
    mcq_exo_difference = pickle.load(filehandle) 

In [18]:
pdm_resid_periods = []
for each, value in enumerate(pdm_periods_resid): 
    if value == 'NaN':
        pdm_resid_periods.append(0)
    else: 
        pdm_resid_periods.append(value)

In [19]:
x = short_periods[0:58]
y =  pdm_resid_periods

source = ColumnDataSource(data=dict(
    x =  short_periods[0:58],
    y = pdm_resid_periods[0:58],
    ls = ls_periods_resid, 
    tess = (cat['tic_id'])[0:58]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

tess_pdm_periods_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="PDM Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='PDM Period', plot_width=900)

tess_pdm_periods_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
tess_pdm_periods_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)
tess_pdm_periods_plot.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
tess_pdm_periods_plot.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")

tess_pdm_periods_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("ls", "$ls"),
    ("TESS", "$tess")] 

show(tess_pdm_periods_plot)

In [20]:
p = figure(plot_width=900, plot_height=600, tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="PDM Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='PDM Period')

base, lower, upper = [], [], []

y = pdm_resid_periods[0:58]
x = short_periods[0:58]

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

for i, per in enumerate(pdm_resid_periods[0:58]):
    lower.append(per - pdm_error[i])
    upper.append(per + pdm_error[i])
    base.append(short_periods[i])
    
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

p.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower", line_color = 'lightsteelblue')
)

p.circle(x = 'x', y = 'y', source = source, color="mediumvioletred", line_color=None)
p.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
p.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
p.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)



show(p)

In [21]:
plot_mcq_err = figure(plot_width=900, plot_height=600, tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="PDM Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='PDM Period')

base, lower, upper = [], [], []

source = ColumnDataSource(data=dict(
    y = pdm_resid_periods[0:58],
    x = short_periods[0:58]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

for i, per in enumerate(pdm_resid_periods[0:58]):
    lower.append(per - mcq_exo_difference[i])
    upper.append(per + mcq_exo_difference[i])
    base.append(short_periods[i])
    
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

plot_mcq_err.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower", line_color = 'lightsteelblue')
)

plot_mcq_err.circle(x = 'x', y = 'y', source = source, color="mediumvioletred", line_color=None)
plot_mcq_err.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
plot_mcq_err.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
plot_mcq_err.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)



show(plot_mcq_err)

In [22]:
ls_resid_periods = []
for each, value in enumerate(ls_periods_resid): 
    if value == 'NaN':
        ls_resid_periods.append(0)
    else: 
        ls_resid_periods.append(value)

In [23]:
x =  short_periods[0:61]
y = ls_periods_resid

source = ColumnDataSource(data=dict(
    x =  short_periods[0:58],
    y = ls_periods_resid, 
    pdm = pdm_periods_resid, 
    tess = (cat['tic_id'])[0:58]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

tess_periods_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="Lomb-Scargle TESS Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='LS TESS Period', plot_width=900)

tess_periods_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
tess_periods_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)
tess_periods_plot.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
tess_periods_plot.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")

tess_periods_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("pdm", "$pdm"),
    ("TESS", "$tess")] 

show(tess_periods_plot)

In [24]:
ls_error = np.std(ls_resid_periods)

In [25]:
p_ls = figure(plot_width=900, plot_height=600, tools="pan,box_zoom,reset,save,hover",
   y_range=[-2, 15], title="LS Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='LS Period')

base, lower, upper = [], [], []

y = ls_resid_periods
x = short_periods[0:61]

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

for i, per in enumerate(ls_resid_periods):
    lower.append(per - ls_error)
    upper.append(per + ls_error)
    base.append(short_periods[i])
    
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

p_ls.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower", line_color = 'lightsteelblue')
)

p_ls.circle(x = 'x', y = 'y', source = source, color="mediumvioletred", line_color=None)
p_ls.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
p_ls.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
p_ls.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)



show(p_ls)

In [26]:
plot_mcq_err_ls = figure(plot_width=900, plot_height=600, tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="LS Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='LS Period')

base, lower, upper = [], [], []

source = ColumnDataSource(data=dict(
    y = ls_resid_periods[0:58],
    x = short_periods[0:58]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

for i, per in enumerate(ls_resid_periods[0:58]):
    lower.append(per - mcq_exo_difference[i])
    upper.append(per + mcq_exo_difference[i])
    base.append(short_periods[i])
    
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

plot_mcq_err_ls.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower", line_color = 'lightsteelblue')
)

plot_mcq_err_ls.circle(x = 'x', y = 'y', source = source, color="mediumvioletred", line_color=None)
plot_mcq_err_ls.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
plot_mcq_err_ls.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
plot_mcq_err_ls.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)



show(plot_mcq_err_ls)

In [27]:
x =  short_periods[0:61]
y = exo_periods_resid

source = ColumnDataSource(data=dict(
    x =  short_periods[0:61],
    y = exo_periods_resid, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

exo_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="Exoplanet ACF TESS Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='Exo ACF TESS Period', plot_width=900)

exo_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
exo_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)
exo_plot.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
exo_plot.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")

exo_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TESS", "$tess")] 

show(exo_plot)

In [28]:
exo_resid_periods = []
for each, value in enumerate(exo_periods_resid): 
    if value == 'NaN':
        exo_resid_periods.append(0)
    else: 
        exo_resid_periods.append(value)

In [29]:
exo_error_std = np.std(exo_resid_periods)

In [30]:
p_ex = figure(plot_width=900, plot_height=600, tools="pan,box_zoom,reset,save,hover",
   y_range=[-2, 15], title="Exo ACF Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='Exo ACF Period')

base, lower, upper = [], [], []

y = exo_resid_periods
x = short_periods[0:61]

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

for i, per in enumerate(exo_resid_periods):
    lower.append(per - exo_error_std)
    upper.append(per + exo_error_std)
    base.append(short_periods[i])
    
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

p_ex.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower", line_color = 'lightsteelblue')
)

p_ex.circle(x = 'x', y = 'y', source = source, color="mediumvioletred", line_color=None)
p_ex.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
p_ex.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
p_ex.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)



show(p_ex)

In [31]:
plot_mcq_err_exo = figure(plot_width=900, plot_height=600, tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="exo Periods from Flux with Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='exo Period')

base, lower, upper = [], [], []

source = ColumnDataSource(data=dict(
    y = exo_resid_periods[0:58],
    x = short_periods[0:58]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

for i, per in enumerate(exo_resid_periods[0:58]):
    lower.append(per - mcq_exo_difference[i])
    upper.append(per + mcq_exo_difference[i])
    base.append(short_periods[i])
    
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

plot_mcq_err_exo.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower", line_color = 'lightsteelblue')
)

plot_mcq_err_exo.circle(x = 'x', y = 'y', source = source, color="mediumvioletred", line_color=None)
plot_mcq_err_exo.line(x, x_half, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
plot_mcq_err_exo.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
plot_mcq_err_exo.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)



show(plot_mcq_err_exo)

In [ ]:
for each difference, check if the difference is less than the error between that and the mcq line. save the plot 
and light curve with an easy to find name. also make lists of stars that fit in the 2x and 1/2x and finally ones that
do not fit on the lines, and save plots somewhere noticeable. Better yet, make a new notebook to do it in! 

run code with less and more filters on the flux - also new notebook

